In [1]:
import os
import sys
stderr = sys.stderr
sys.stderr = open(os.devnull, 'w')
sys.stderr = stderr

import pandas as pd
from sklearn.utils import shuffle
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv("sets_de_datos/train.csv")
descripciones_features = pd.read_csv('data/descripciones_chi2.csv', index_col = 0)
df = df.join(descripciones_features, on = "id")
lat_long = pd.read_csv("sets_de_datos/train_final1.csv").astype('float64')
df["precio"] /= 10000
df[["lat","lng"]] = lat_long[["lat","lng"]]
df_precios_por_tipo = df.groupby('tipodepropiedad').agg({'precio':'mean'}).rename(columns={'precio':'precio_por_tipo'})
df

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,centroscomercialescercanos,precio,0,1,2,3,4,5,6,7
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,0.0,227.3,0,0,0,0,0,0,0,0
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,1.0,360.0,0,2,0,2,1,1,1,0
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,0.0,120.0,0,0,0,0,0,0,0,0
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,1.0,65.0,1,0,1,0,0,0,0,0
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,0.0,115.0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,119879,bonita casas de 2 recamaras a 10 minutos del c...,vendo casa en bosques de ica residencial a 10 ...,Casa,BOSQUES,Zinacantepec,Edo. de México,0.0,2.0,2.0,...,0.0,65.0,0,0,0,0,0,0,0,0
239996,259178,casa en condominio a 10 min. del centro de toluca,"casa con un jardin amplio, un cuarto de servic...",Casa,Filiberto Navas 325,Toluca,Edo. de México,0.0,3.0,3.0,...,1.0,194.0,0,2,0,2,2,1,1,0
239997,131932,nicolas san juan,"departamento con excelente ubicación, muy cerc...",Apartamento,Nicolas San Juan,Benito Juárez,Distrito Federal,20.0,2.0,1.0,...,0.0,340.0,0,2,1,1,2,1,1,0
239998,146867,casa sola. javier rojo gomez.,"casa sola, dividida en cuatro departamentos de...",Casa,Javier Rojo Gomez 120,Iztapalapa,Distrito Federal,20.0,4.0,0.0,...,1.0,289.0,0,0,0,0,0,0,0,0


In [3]:
def limpiar(df):
    df.garages = df.garages.fillna(0)
    df.metroscubiertos = df.metroscubiertos.fillna(df.metroscubiertos.mean())
    df.antiguedad = df.antiguedad.fillna(df.antiguedad.mean())
    df.banos = df.banos.fillna(1)
    df.habitaciones = df.habitaciones.fillna(df.habitaciones.mean())
    df.tipodepropiedad = df.tipodepropiedad.fillna('Casa')
    df.metrostotales = df.metrostotales.fillna(0)
    df['metroscubiertos'] = df['metroscubiertos'].fillna(df['metroscubiertos'].mean())
    df.ciudad = df.ciudad.fillna("")

def nuevas_features(df, precios_tipo,precio_m2,promedios,default_m2):
    df['ratio_cubierto'] = df.apply(lambda x: x['metroscubiertos']/x['metrostotales'] if x['metrostotales'] else 1, axis = 1)
    df['tipodepropiedad'] = df['tipodepropiedad'].apply(lambda x: precios_tipo.loc[x]['precio_por_tipo'])
    df['precio_x_m2'] = df.apply(lambda x: precio_x_m2.get(x['ciudad'],default_m2), axis = 1)
    df['mean_hab'] = df.apply(lambda x: promedios['mean_hab'].get(x['ciudad'],default_m2), axis = 1)
    df['mean_ban'] = df.apply(lambda x: promedios['mean_ban'].get(x['ciudad'],default_m2), axis = 1)
    df['mean_gar'] = df.apply(lambda x: promedios['mean_gar'].get(x['ciudad'],default_m2), axis = 1)

def evaluar_rf(modelo, X_test, y_test):
    y_pred = modelo.predict(X_test)
    errors = abs(y_pred - y_test)
    mape = 100 * np.mean(errors / y_test)
    accuracy = 100 - mape
    print('Performance del modelo:')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy
    
df_precios_por_tipo

,precio_por_tipo
tipodepropiedad,
Apartamento,276.376866
Bodega comercial,269.375770
Casa,239.815771
Casa en condominio,289.892704
Casa uso de suelo,373.246923
Departamento Compartido,256.016189
Duplex,127.175797
Edificio,500.949937
Garage,50.000000


In [4]:
limpiar(df)
df['precio_x_m2'] = df['precio']/df['metroscubiertos']
precio_x_m2 = df.groupby('ciudad').agg({'precio_x_m2':'mean'}).to_dict()['precio_x_m2']
default = df.groupby('ciudad').agg({'precio_x_m2':'mean'})['precio_x_m2'].mean()
promedios = df.set_index('ciudad')\
            .join(df.groupby('ciudad')\
                  .agg({'habitaciones':'mean', 'garages':'mean', 'banos':'mean'})\
                      .rename(columns={'habitaciones':'mean_hab', 'banos':'mean_ban', 'garages':'mean_gar'}))\
                        [['mean_hab','mean_gar','mean_ban']].to_dict()
nuevas_features(df, df_precios_por_tipo, precio_x_m2,promedios,default)

In [5]:
FEATURES = ['0', '1', '2', '3', '4', '5', '6', '7', 'tipodepropiedad', 'lat', 'lng', 'garages', 'habitaciones', 'antiguedad', 'metroscubiertos', 'banos', 'ratio_cubierto', 'mean_hab','mean_ban', 'mean_gar', 'precio_x_m2']
df = shuffle(df)
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
target_train = pd.DataFrame(train["precio"])
test = df[~msk]
target_test = pd.DataFrame(test["precio"])
train = train.drop(columns = ["id", "precio"])
test = test.drop(columns = ["id", "precio"])

In [6]:
params = {'max_depth': 51, 'max_features': 9, 'min_samples_leaf': 2, 'min_samples_split': 9, 'n_estimators': 226}

In [7]:
rf = RandomForestRegressor(**params)

In [8]:
rf.fit(train[FEATURES], target_train.values.ravel())

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=51,
                      max_features=9, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=9,
                      min_weight_fraction_leaf=0.0, n_estimators=226,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [9]:
evaluar_rf(rf, test[FEATURES], target_test.values.ravel())

Performance del modelo:
Average Error: 61.1870 degrees.
Accuracy = 70.61%.


70.60929279384618

In [10]:
rf.predict(test[FEATURES])

array([589.86631921, 210.73251332,  90.88816995, ..., 142.88584286,
        95.12650461, 262.58176528])

In [11]:
for i, x in enumerate(rf.feature_importances_):
  print(FEATURES[i], x)

0 0.0015273000495917963
1 0.007655783947306141
2 0.004287758120692283
3 0.003608088483235405
4 0.0029879553714684043
5 0.023468422860470045
6 0.01662053136179842
7 0.0009750549264534195
tipodepropiedad 0.04257902547821982
lat 0.03850706714513427
lng 0.026361609361191293
garages 0.02040661463427365
habitaciones 0.02354269480266241
antiguedad 0.030043233742270248
metroscubiertos 0.33550385483787826
banos 0.07797202914007767
ratio_cubierto 0.06017892392802834
mean_hab 0.026724770309743597
mean_ban 0.016765729386512003
mean_gar 0.015488840927357937
precio_x_m2 0.22479471118563454


In [12]:
test_kaggle = pd.read_csv('sets_de_datos/test.csv', index_col = 0)
lat_long_kaggle = pd.read_csv('sets_de_datos/test_final1.csv', index_col = 0)
descripciones_kaggle = pd.read_csv('data/descripciones_chi2_kaggle.csv', index_col = 0)
test_kaggle[['lat','lng']] = lat_long_kaggle[['lat','lng']]
test_kaggle = test_kaggle.join(descripciones_kaggle, on='id')

In [13]:
precio_x_m2 = df.groupby('ciudad').agg({'precio_x_m2':'mean'}).to_dict()['precio_x_m2']
default = df.groupby('ciudad').agg({'precio_x_m2':'mean'})['precio_x_m2'].mean()
limpiar(test_kaggle)
nuevas_features(test_kaggle, df_precios_por_tipo, precio_x_m2, promedios, default)

In [14]:
test_kaggle

,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,banos,...,3,4,5,6,7,ratio_cubierto,precio_x_m2,mean_hab,mean_ban,mean_gar
id,,,,,,,,,,,,,,,,,,,,,
4941,"casa en venta en miguel hidalgo, distrito federal",<p>excelente casa estilo moderno.</p>,239.815771,Bosque de Cedros,Miguel Hidalgo,Distrito Federal,29.000000,3.0,0.0,4.0,...,0,0,0,0,0,1.000000,3.153119,2.508828,1.866091,1.303192
51775,departamentos en venta en montebello,<p>departamento una recamara:\n</p><p>departam...,276.376866,NaN,Mérida,Yucatán,8.152599,1.0,1.0,1.0,...,0,0,0,0,0,1.000000,0.962266,2.928739,2.425859,1.357163
115253,departamento nuevo delegación coyoacán de 87 m...,"departamento nuevo de 87.06 m2, 1 cajón de est...",276.376866,"Pueblo de los Reyes, Coyoacán, Mexico D.F.",Coyoacán,Distrito Federal,0.000000,2.0,1.0,2.0,...,0,0,0,0,0,0.870000,2.049670,3.025226,1.979029,1.336671
299321,departamento en venta en acapulco,<p> raíces dv001 precioso departamento tipo k...,276.376866,NaN,Acapulco de Juárez,Guerrero,2.000000,2.0,2.0,2.0,...,0,0,0,0,0,1.000000,1.543191,2.823817,2.198452,1.224531
173570,bonita casa sola equipada de dos niveles en lo...,"<p>casa sola, bonita de dos rec&aacute;maras u...",239.815771,CEDROS,Tultitlán,Edo. de México,10.000000,2.0,1.0,1.0,...,0,0,0,0,0,1.052632,0.869196,2.799797,1.340136,1.073858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75094,oportunidad!! se vende amplia casa en col. moc...,oportunidad!! ideal para oficina o casa habita...,239.815771,Oriente 172 # 265,Venustiano Carranza,Distrito Federal,20.000000,4.0,3.0,3.0,...,0,0,0,0,0,1.000000,1.582998,2.824782,1.497598,0.849005
171847,colinas de ecatepec,"casa, sala comedor, patio de servicio, buenas ...",239.815771,colinas,Ecatepec de Morelos,Edo. de México,10.000000,3.0,1.0,2.0,...,0,0,0,0,0,0.816092,0.872189,2.951562,1.503011,1.185734
138313,estrene hermosa casa en sierra morena,hermosa casa lista para habitarse ubicada en f...,239.815771,s/calle,Guadalupe,Nuevo León,5.000000,3.0,2.0,2.0,...,0,0,0,0,0,1.000000,0.877376,2.973335,1.864879,1.074531


In [15]:
submit = rf.predict(test_kaggle[FEATURES])
submit_df = pd.DataFrame({"id": test_kaggle.index, "target": submit*10000})
submit_df

,id,target
0,4941,7.237950e+06
1,51775,8.637204e+05
2,115253,2.499219e+06
3,299321,1.169020e+06
4,173570,6.384593e+05
...,...,...
59995,75094,3.513068e+06
59996,171847,7.383511e+05
59997,138313,8.468142e+05
59998,271268,1.557227e+06


In [16]:
submit_df.to_csv("submits/submit_desc2.csv", index=False)

In [17]:
for i, x in enumerate(rf.feature_importances_):
  try:
    print(FEATURES[i],x)
  except:
    print(i - 9, x)

0 0.0015273000495917963
1 0.007655783947306141
2 0.004287758120692283
3 0.003608088483235405
4 0.0029879553714684043
5 0.023468422860470045
6 0.01662053136179842
7 0.0009750549264534195
tipodepropiedad 0.04257902547821982
lat 0.03850706714513427
lng 0.026361609361191293
garages 0.02040661463427365
habitaciones 0.02354269480266241
antiguedad 0.030043233742270248
metroscubiertos 0.33550385483787826
banos 0.07797202914007767
ratio_cubierto 0.06017892392802834
mean_hab 0.026724770309743597
mean_ban 0.016765729386512003
mean_gar 0.015488840927357937
precio_x_m2 0.22479471118563454


In [18]:
FEATURES = ['tipodepropiedad', 'lat', 'lng', 'garages', 'habitaciones', 'antiguedad', 'metroscubiertos', 'banos', 'ratio_cubierto', 'mean_hab','mean_ban', 'mean_gar', 'precio_x_m2','gimnasio','piscina']
